## Data Collect - ID - Name - Price

### One Product

In [ ]:
#import pandas as pd
#import numpy as np
#from bs4 import BeautifulSoup
#import requests

#url = 'https://www2.hm.com/en_us/men/products/jeans.html'
#headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
#requests = requests.get(url, headers=headers)
#page = requests.text
#soup = BeautifulSoup(page, 'html.parser')


#ID - Title - Price

#product_id = soup.find('article',class_='hm-product-item')['data-articlecode']
#
#product_name = soup.find('div', class_="image-container").find('a')['title']
#
#product_price = soup.find('strong',class_="item-price").find('span').get_text()
#product_price = pd.to_numeric(list(filter(None,product_price.split('$'))))
#
#

### Multiple Products

#### Imports

In [940]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

#### Collect

In [938]:
#Coletar número total de products por página (vitrine) e número total de products para calcular número de páginas e 
#quantos espaços na vitrine (total_show)


url = 'https://www2.hm.com/en_us/men/products/jeans.html'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
requests = requests.get(url, headers=headers)
page = requests.text

soup = BeautifulSoup(page, 'html.parser')
total_products = soup.find('h2',class_="load-more-heading")['data-total']
items_pages = soup.find('h2',class_="load-more-heading")['data-items-shown']
pagination = round(int(total_products)/int(items_pages))
total_show = int(items_pages)*pagination

In [941]:
#Criando listas vazias e iterador
i = int(items_pages)
aux1 = []
aux2 = []
aux3 = []

#iteração que permite coletar todos os produtos de todas as páginas
while i <= int(total_show):
    
    if i <= (i*pagination):
        
        url02 = 'https://www2.hm.com/en_us/men/products/jeans.html?offset=0&page-size='+str(i)
        
        #API Request
        requests_2 = requests.get(url02, headers=headers)
        page_2 = requests_2.text
        soup_2 = BeautifulSoup(page_2, 'html.parser')
        
        number_products = soup_2.find_all('li',class_='product-item')
    
        products_id = [soup_2.find_all('article',class_='hm-product-item')[j]['data-articlecode'] for j in range(len(number_products))]
        aux1 = aux1 + products_id 

        name_products = [soup_2.find_all('div', class_="image-container")[k].find('a')['title'] for k in range(len(number_products))]
        aux2 = aux2 + name_products
        
        price_products = [list(filter(None,soup_2.find_all('strong',class_="item-price")[u].find('span').get_text().split('$')))for u in range(len(number_products))]
        aux3 = aux3 + price_products
            
            #price_products = pd.DataFrame(price_products)
        
        i = int(items_pages) + i
        print(url02)

    else:
        print('fim')



https://www2.hm.com/en_us/men/products/jeans.html?offset=0&page-size=36
https://www2.hm.com/en_us/men/products/jeans.html?offset=0&page-size=72
https://www2.hm.com/en_us/men/products/jeans.html?offset=0&page-size=108


In [1119]:
product_id=pd.DataFrame(aux1)
product_name=pd.DataFrame(aux2)
product_price=pd.DataFrame(aux3)

In [1127]:
df1 = pd.concat([product_id,product_name],axis=1)
data = pd.concat([df1,product_price],axis=1)
data.columns = ['product_id','product_name','product_price']

In [1128]:
#data.reset_index(drop=True,inplace=True)
data.head()

,product_id,product_name,product_price
0,0690449043,Skinny Jeans,39.99
1,1004199003,Skinny Cropped Jeans,29.99
2,0690449051,Skinny Jeans,39.99
3,0690449056,Skinny Jeans,39.99
4,1004199005,Skinny Cropped Jeans,29.99


In [1129]:
len(data)

206

In [1130]:
data=data.drop_duplicates(subset='product_id', keep="first")

In [1131]:
len(data)

98

In [1132]:
#Separate product_id in style and color id:
data['style_id'] = data['product_id'].apply( lambda x: x[:-3])
data['color_id'] = data['product_id'].apply( lambda x: x[-3:])

In [1133]:
#drop the style id duplicates to get the product color only one time to either product_id

data=data.drop_duplicates(subset='style_id', keep="first")

In [1134]:
len(data)

28

In [1159]:
data = data.loc[data['product_id']!='0811993034']

In [1160]:
data  = data.reset_index(drop=True)
len(data)

27

In [1161]:
data.loc[16,'product_id']

'1024711001'

In [1267]:
data.head()

,product_id,product_name,product_price,style_id,color_id
0,0690449043,Skinny Jeans,39.99,0690449,043
1,1004199003,Skinny Cropped Jeans,29.99,1004199,003
2,1024256001,Slim Jeans,19.99,1024256,001
3,0985159001,Skinny Jeans,19.99,0985159,001
4,0938875012,Slim Tapered Jeans,39.99,0938875,012


## Data Collect - Composition - Collor

### One Product

In [722]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [1174]:
#API Requests

url = 'https://www2.hm.com/en_us/productpage.1024256001.html'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
requests_3 = requests.get(url,headers = headers)
page_3 = requests_3.text
soup_3 = BeautifulSoup(page_3,'html.parser')


In [1102]:
#Get Collor

list_item = soup_3.find('div',class_='mini-slider').find_all('li',class_="list-item")
color_item = [list_item[i].find('a')['data-color'] for i in range(len(list_item))]
code_item = [list_item[i].find('a')['data-articlecode'] for i in range(len(list_item))]
color_item = pd.DataFrame(color_item)
code_item = pd.DataFrame(code_item)
item_id = pd.concat([color_item,code_item],axis=1)

In [1104]:
 len(list_item)

8

In [1110]:
item_id

,0,0
0,Light denim blue,1004199001
1,Black,1004199002
2,Denim blue,1004199003
3,White,1004199004
4,Light gray,1004199005


In [922]:
#soup_3.find('div',class_='mini-slider')

In [1188]:
#Get Composition

composition_list = soup_3.find_all('div',class_="details parbase")[0].find_all('script')[1].text
composition_list = composition_list.replace('\r', '').replace('\t', '').replace('\n', '').replace('\\"', '').replace("\\'","")
p = re.search('\[.*(s*{.*}s*).*\]', composition_list).group(0)


# needs to be " " to the json.loads work
p = p.replace("'", '"')
p = p.replace('title', '"title"')
p = p.replace('values', '"values"')
json_data = json.loads(p)
composition = pd.DataFrame.from_dict(json_data)
df_composition = (composition).T
df_composition.columns = df_composition.iloc[:1].iloc[0,:].tolist()
df_composition = df_composition.reset_index(drop=True)
df_composition=df_composition.drop(0)
#pd.to_numeric(df_composition.iloc[0,4])
#df_composition.iloc[0,4] = pd.to_numeric(df_composition.iloc[0,4][0])
for i in range(len(df_composition.columns)-1):
    df_composition.iloc[0,i] = df_composition.iloc[0,i][0]

In [1184]:
df_composition

,Size,Fit,Composition,Art. No.
1,The model is 187cm/62 and wears a size 32/32,Skinny fit,"Cotton 98%, Spandex 2%",[0690449043]


### Multiple Products

In [926]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [927]:
#API Requests

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}


In [1162]:
#Get Collor

aux = pd.DataFrame()

for i in range(len(data)):
    url = 'https://www2.hm.com/en_us/productpage.'+data.loc[i,'product_id']+'.html'
    requests_3 = requests.get(url,headers = headers)
    page_3 = requests_3.text
    soup_3 = BeautifulSoup(page_3,'html.parser')
    list_item = soup_3.find('div',class_='mini-slider').find_all('li',class_="list-item")
    color_item = [list_item[i].find('a')['data-color'] for i in range(len(list_item))]
    code_item = [list_item[i].find('a')['data-articlecode'] for i in range(len(list_item))]
    color_name = pd.DataFrame(color_item)
    code_item = pd.DataFrame(code_item)
    item_id = pd.concat([color_name,code_item],axis=1)
    aux = pd.concat([item_id,aux],axis=0)
    print(data.loc[i,'product_id'])

0690449043
1004199003
1024256001
0985159001
0938875012
1008549006
0927964002
0979945001
1027852007
1013317008
1008110001
0875105024
0751994024
0974597002
1018704001
0730863033
1024711001
0427159006
0985197003
1049466001
1004476006
0974202003
1028865001
1025726002
0993887001
1048642001
0986779002


In [1167]:
aux.columns = ['name_color','product_id']

In [1170]:
df_color = aux.copy()

In [1171]:
df_color['style_id'] = df_color['product_id'].apply( lambda x: x[:-3])
df_color['color_id'] = df_color['product_id'].apply( lambda x: x[-3:])

In [1172]:
df_color.head()

,name_color,product_id,style_id,color_id
0,Denim blue,0986779002,0986779,002
1,Black,0986779003,0986779,003
2,Dark denim blue,0986779004,0986779,004
3,Denim blue,0986779005,0986779,005
0,Denim blue,1048642001,1048642,001


In [1260]:
#Get Composition

aux_2 = pd.DataFrame()

for i in range(len(data)):
    url = 'https://www2.hm.com/en_us/productpage.'+data.loc[i,'product_id']+'.html'
    requests_3 = requests.get(url,headers = headers)
    page_3 = requests_3.text
    soup_3 = BeautifulSoup(page_3,'html.parser')
    composition_list = soup_3.find_all('div',class_="details parbase")[0].find_all('script')[1].text
    composition_list = composition_list.replace('\r', '').replace('\t', '').replace('\n', '').replace('\\"', '').replace("\\'","")
    p = re.search('\[.*(s*{.*}s*).*\]', composition_list).group(0)
    # needs to be " " to the json.loads work
    p = p.replace("'", '"')
    p = p.replace('title', '"title"')
    p = p.replace('values', '"values"')
    json_data = json.loads(p)
    composition = pd.DataFrame.from_dict(json_data)
    df_composition = (composition).T
    df_composition.columns = df_composition.iloc[:1].iloc[0,:].tolist()
    df_composition = df_composition.reset_index(drop=True)
    df_composition=df_composition.drop(0)
    for i in range(len(df_composition.columns)):
        df_composition.iloc[0,i] = df_composition.iloc[0,i][0]
    
    aux_2 = pd.concat([aux_2,df_composition],axis=0)
    

In [1261]:
aux_2 = aux_2.reset_index(drop=True)

In [1262]:
aux_2.head()

,Size,Fit,Composition,Art. No.,More sustainable materials
0,The model is 187cm/62 and wears a size 32/32,Skinny fit,"Cotton 98%, Spandex 2%",0690449043,NaN
1,The model is 185cm/61 and wears a size 31,Skinny fit,"Pocket lining: Polyester 65%, Cotton 35%",1004199003,Shell: Recycled cotton 20%
2,The model is 185cm/61 and wears a size 31/32,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",1024256001,Lining: Recycled cotton 20%
3,The model is 187cm/62 and wears a size 32/32,Skinny fit,"Shell: Cotton 99%, Spandex 1%",0985159001,Shell: Recycled cotton 20%
4,The model is 185cm/61 and wears a size 31/32,Slim fit,"Cotton 99%, Spandex 1%",0938875012,Shell: Recycled cotton 20%


In [1265]:
aux_2['style_id'] = aux_2['Art. No.'].apply(lambda x: x[:-3])
aux_2['color_id'] = aux_2['Art. No.'].apply(lambda x: x[-3:])


In [1266]:
aux_2.head()

,Size,Fit,Composition,Art. No.,More sustainable materials,style_id,color_id
0,The model is 187cm/62 and wears a size 32/32,Skinny fit,"Cotton 98%, Spandex 2%",0690449043,NaN,0690449,043
1,The model is 185cm/61 and wears a size 31,Skinny fit,"Pocket lining: Polyester 65%, Cotton 35%",1004199003,Shell: Recycled cotton 20%,1004199,003
2,The model is 185cm/61 and wears a size 31/32,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",1024256001,Lining: Recycled cotton 20%,1024256,001
3,The model is 187cm/62 and wears a size 32/32,Skinny fit,"Shell: Cotton 99%, Spandex 1%",0985159001,Shell: Recycled cotton 20%,0985159,001
4,The model is 185cm/61 and wears a size 31/32,Slim fit,"Cotton 99%, Spandex 1%",0938875012,Shell: Recycled cotton 20%,0938875,012


In [1269]:
df_details = aux_2.copy()

In [1284]:
data = data.drop(columns=['product_id','color_id'])

In [1285]:
df1 = data.merge(df_color,how='inner', on='style_id' )
df1

,product_name,product_price,style_id,name_color,product_id,color_id
0,Skinny Jeans,39.99,0690449,Light denim blue/trashed,0690449001,001
1,Skinny Jeans,39.99,0690449,Denim blue,0690449002,002
2,Skinny Jeans,39.99,0690449,Black/washed,0690449006,006
3,Skinny Jeans,39.99,0690449,Light denim blue,0690449007,007
4,Skinny Jeans,39.99,0690449,Black washed out,0690449009,009
...,...,...,...,...,...,...
174,Regular Bootcut Jeans,39.99,1048642,Denim black,1048642002,002
175,Relaxed Tapered Pull-on Jeans,29.99,0986779,Denim blue,0986779002,002
176,Relaxed Tapered Pull-on Jeans,29.99,0986779,Black,0986779003,003
177,Relaxed Tapered Pull-on Jeans,29.99,0986779,Dark denim blue,0986779004,004


In [1286]:
df_details = df_details.drop(columns=['color_id','Art. No.'])

In [1291]:
df = df1.merge(df_details,how='inner',on='style_id')
df = df[['product_name','product_price','product_id','style_id','color_id','name_color','Size','Fit','Composition','More sustainable materials']]
df

,product_name,product_price,product_id,style_id,color_id,name_color,Size,Fit,Composition,More sustainable materials
0,Skinny Jeans,39.99,0690449001,0690449,001,Light denim blue/trashed,The model is 187cm/62 and wears a size 32/32,Skinny fit,"Cotton 98%, Spandex 2%",NaN
1,Skinny Jeans,39.99,0690449002,0690449,002,Denim blue,The model is 187cm/62 and wears a size 32/32,Skinny fit,"Cotton 98%, Spandex 2%",NaN
2,Skinny Jeans,39.99,0690449006,0690449,006,Black/washed,The model is 187cm/62 and wears a size 32/32,Skinny fit,"Cotton 98%, Spandex 2%",NaN
3,Skinny Jeans,39.99,0690449007,0690449,007,Light denim blue,The model is 187cm/62 and wears a size 32/32,Skinny fit,"Cotton 98%, Spandex 2%",NaN
4,Skinny Jeans,39.99,0690449009,0690449,009,Black washed out,The model is 187cm/62 and wears a size 32/32,Skinny fit,"Cotton 98%, Spandex 2%",NaN
...,...,...,...,...,...,...,...,...,...,...
174,Regular Bootcut Jeans,39.99,1048642002,1048642,002,Denim black,NaN,Regular fit,"Shell: Cotton 99%, Spandex 1%",Lining: Recycled polyester 80%
175,Relaxed Tapered Pull-on Jeans,29.99,0986779002,0986779,002,Denim blue,NaN,Relaxed fit,"Pocket lining: Polyester 65%, Cotton 35%","Lining: Recycled polyester 65%, Recycled cotto..."
176,Relaxed Tapered Pull-on Jeans,29.99,0986779003,0986779,003,Black,NaN,Relaxed fit,"Pocket lining: Polyester 65%, Cotton 35%","Lining: Recycled polyester 65%, Recycled cotto..."
177,Relaxed Tapered Pull-on Jeans,29.99,0986779004,0986779,004,Dark denim blue,NaN,Relaxed fit,"Pocket lining: Polyester 65%, Cotton 35%","Lining: Recycled polyester 65%, Recycled cotto..."


In [1293]:
df['product_name'].unique()

array(['Skinny Jeans', 'Skinny Cropped Jeans', 'Slim Jeans',
       'Slim Tapered Jeans', 'Regular Jeans',
       'Regular Tapered Crop Jeans', 'Loose Jeans',
       'Relaxed Denim Joggers', 'Hybrid Regular Tapered Joggers',
       'Freefit® Slim Jeans', 'Relaxed Jeans', 'Relaxed Pull-on Jeans',
       'Trashed Skinny Jeans', 'Loose Carpenter Jeans',
       'Regular Denim Joggers', 'Relaxed Jeans with Embroidery Detail',
       'Hybrid Regular Denim Joggers', 'Regular Bootcut Jeans',
       'Relaxed Tapered Pull-on Jeans'], dtype=object)

In [1295]:
median_by_type = df[['product_name','product_price']].groupby('product_name').median()
median_by_type

,product_price
product_name,
Freefit® Slim Jeans,49.99
Hybrid Regular Denim Joggers,39.99
Hybrid Regular Tapered Joggers,39.99
Loose Carpenter Jeans,39.99
Loose Jeans,39.99
Regular Bootcut Jeans,39.99
Regular Denim Joggers,29.99
Regular Jeans,19.99
Regular Tapered Crop Jeans,19.99
